A partir des cartes océrisées et indexées à partir du script, on merge les résultats avec les métadonnées qu'on a scrappé au préalable.

Géoréférencement avec l'API Nominatim



# GOOGLE COLAB !!!

In [ ]:
import pandas as pd
import requests
import time
from google.colab import drive
import os

# --- CONFIGURATION ---
drive.mount('/content/drive')
input_csv = '/content/drive/MyDrive/vaucluse/output/metadata_output.csv'
output_csv = '/content/drive/MyDrive/vaucluse/output/vaucluse_georef_nominatim.csv'

# Cache pour économiser l'API Nominatim
cache_nominatim = {}

def call_nominatim(query):
    """Interroge Nominatim et met en cache le résultat"""
    if not query or query in cache_nominatim:
        return cache_nominatim.get(query), True

    time.sleep(1.1)  # Respect de la limite de 1 requête / seconde
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": query, "format": "json", "limit": 1, "addressdetails": 1}
    headers = {"User-Agent": "ProjetCartoVaucluse_Funnel_CSV/1.0"}

    try:
        response = requests.get(url, params=params, headers=headers)
        res_json = response.json()
        if res_json:
            result = {
                "lat": float(res_json[0]["lat"]),
                "lon": float(res_json[0]["lon"]),
                "display_name": res_json[0]["display_name"]
            }
            cache_nominatim[query] = result
            return result, False
    except Exception as e:
        print(f"      ⚠️ Erreur réseau : {e}")

    cache_nominatim[query] = None
    return None, False

# --- CHARGEMENT ---
df = pd.read_csv(input_csv)
print(f"Lancement du géocodage sur {len(df)} cartes...\n")

results_coords = []
results_adresses = []
results_precision = []

# --- BOUCLE DE TRAITEMENT ---
for i, row in df.iterrows():
    city = str(row.get('trans_city', ''))
    monument = str(row.get('trans_monument_uniformise', ''))
    edifice_raw = str(row.get('Édifice', ''))
    hamlet = str(row.get('trans_hamlet_uniformise', ''))

    # Nettoyage de l'édifice (on prend ce qui est après le "--")
    edifice_clean = edifice_raw.split('--')[-1].strip() if '--' in edifice_raw else edifice_raw.strip()

    print(f"[{i+1}/{len(df)}] Analyse : {row['File name']} ({city})")

    final_res = None
    source = "commune (fallback)"

    # --- ÉTAPE 1 : Monument uniformisé ---
    if monument and "Aucun" not in monument and monument != "nan":
        query = f"{monument}, {city}, 84, France"
        final_res, _ = call_nominatim(query)
        if final_res: source = "monument_uniformise"

    # --- ÉTAPE 2 : Édifice (si étape 1 échoue) ---
    if not final_res and edifice_clean and edifice_clean != "nan":
        query = f"{edifice_clean}, {city}, 84, France"
        final_res, _ = call_nominatim(query)
        if final_res: source = "edifice"

    # --- ÉTAPE 3 : Hameau (si étape 2 échoue) ---
    if not final_res and hamlet and "Aucun" not in hamlet and hamlet != "nan":
        query = f"{hamlet}, {city}, 84, France"
        final_res, _ = call_nominatim(query)
        if final_res: source = "hameau"

    # --- ÉTAPE 4 : Latitude/Longitude existantes (Fallback) ---
    if final_res:
        coords_str = f"{final_res['lat']}, {final_res['lon']}"
        adresse = final_res['display_name']
        print(f"   ✅ Trouvé via {source}")
    else:
        # On utilise les colonnes existantes du CSV
        lat_orig = row.get('latitude')
        lon_orig = row.get('longitude')
        coords_str = f"{lat_orig}, {lon_orig}"
        adresse = "" # Pas d'adresse précise si on est au centre-ville
        print(f"   📍 Repli sur les coordonnées de la commune")

    results_coords.append(coords_str)
    results_adresses.append(adresse)
    results_precision.append(source)

# --- ENREGISTREMENT ---
df['coordonnees_georeferencement'] = results_coords
df['adresse_precise'] = results_adresses
df['source_georef'] = results_precision # Optionnel : pour savoir quel niveau a fonctionné

df.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"\n✅ Géocodage terminé ! Fichier sauvegardé : {output_csv}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 Lancement du géocodage sur 84 fiches...

[1/84] Analyse : FRAD084_07FI003_0027.jpg (Apt)
   ✅ Trouvé via hameau
[2/84] Analyse : FRAD084_07FI003_0073.jpg (Apt)
   ✅ Trouvé via edifice
[3/84] Analyse : FRAD084_07FI007_0940.jpg (Avignon)
   ✅ Trouvé via monument_uniformise
[4/84] Analyse : FRAD084_07FI007_0819.jpg (nan)
   📍 Repli sur les coordonnées de la commune
[5/84] Analyse : FRAD084_07FI007_0699.jpg (nan)
   📍 Repli sur les coordonnées de la commune
[6/84] Analyse : FRAD084_07FI007_1063_01.jpg (Avignon)
   ✅ Trouvé via edifice
[7/84] Analyse : FRAD084_07FI007_0375_01.jpg (Avignon)
   ✅ Trouvé via monument_uniformise
[8/84] Analyse : FRAD084_07FI007_0382.jpg (Avignon)
   ✅ Trouvé via monument_uniformise
[9/84] Analyse : FRAD084_07FI007_0950.jpg (Avignon)
   ✅ Trouvé via edifice
[10/84] Analyse : FRAD084_07FI007_0965.jpg (Avignon)
   ✅ Trouvé via monument

Construction de la carte avec Folium

In [1]:
########### CONSTRUCTION DE LA CARTE ##############


from google.colab import drive
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import os
import base64
import re
import requests

# --- 1. CONFIGURATION & CHARTE ---
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/vaucluse/output/vaucluse_georef_nominatim.csv'
images_dir = '/content/drive/MyDrive/vaucluse/data'
output_html = 'carte_vaucluse_interactive.html'

COLOR_GREEN = "#046B58"  # Vert classique
COLOR_ORANGE = "#E45955" # Orange classique

# --- 2. FONCTIONS UTILES ---

def extract_year(date_str):
    """Extrait l'année d'une chaîne type '[vers 1900]'"""
    years = re.findall(r'\d{4}', str(date_str))
    return int(years[0]) if years else 9999

def get_base64_image(img_name):
    """Encode l'image en base64 pour l'inclure directement dans le HTML"""
    img_path = os.path.join(images_dir, img_name)
    if os.path.exists(img_path):
        try:
            with open(img_path, "rb") as f:
                encoded = base64.b64encode(f.read()).decode()
                return f"data:image/jpeg;base64,{encoded}"
        except: return None
    return None

# --- 3. PRÉPARATION DES DONNÉES ---
df = pd.read_csv(file_path)
df[['lat', 'lon']] = df['coordonnees_georeferencement'].str.split(',', expand=True).astype(float)
df['year'] = df['Dates'].apply(extract_year)
df = df.dropna(subset=['lat', 'lon'])

# --- 4. INITIALISATION DE LA CARTE ---
m = folium.Map(location=[43.92, 5.2], zoom_start=9, tiles='OpenStreetMap')

# 4.1. Chargement Géographique Robuste
dept_url = "https://geo.api.gouv.fr/departements/84?format=geojson&geometry=contour"
comm_url = "https://geo.api.gouv.fr/departements/84/communes?format=geojson&geometry=contour"

try:
    vaucluse_geojson = requests.get(dept_url).json()
    communes_geojson = requests.get(comm_url).json()

    # Extraction de la géométrie
    v_feat = vaucluse_geojson['features'][0] if 'features' in vaucluse_geojson else vaucluse_geojson
    v_geom = v_feat['geometry']

    # 4.2. Création du Masque (Focus Vaucluse)
    # Rectangle mondial
    world_ring = [[-180, -90], [180, -90], [180, 90], [-180, 90], [-180, -90]]
    mask_coords = [world_ring]

    # Ajout des "trous" (rings du Vaucluse)
    if v_geom['type'] == 'Polygon':
        mask_coords.append(v_geom['coordinates'][0])
    elif v_geom['type'] == 'MultiPolygon':
        for poly in v_geom['coordinates']:
            mask_coords.append(poly[0])

    folium.GeoJson(
        {"type": "Feature", "geometry": {"type": "Polygon", "coordinates": mask_coords}},
        style_function=lambda x: {'fillColor': '#000000', 'color': 'none', 'fillOpacity': 0.5},
        interactive=False
    ).add_to(m)

    # 4.3. Frontières
    # Département (Gros trait vert)
    folium.GeoJson(
        vaucluse_geojson,
        style_function=lambda x: {'color': COLOR_GREEN, 'weight': 6, 'fillOpacity': 0},
        interactive=False
    ).add_to(m)

    # Communes (Trait gris fin)
    folium.GeoJson(
        communes_geojson,
        style_function=lambda x: {'color': '#666666', 'weight': 0.4, 'fillOpacity': 0},
        interactive=False
    ).add_to(m)

except Exception as e:
    print(f"Erreur chargement contours : {e}")

# --- 5. CLUSTERS ET MARQUEURS ---

# Tri chronologique des périodes
all_years = sorted([y for y in df['year'].unique() if y != 9999])
decades_keys = sorted(list(set([(y // 10) * 10 for y in all_years])))
if 9999 in df['year'].values: decades_keys.append(9999)

groups = {}
for d in decades_keys:
    label = f"Années {d}" if d != 9999 else "Date inconnue"
    # MarkerCluster par décennie pour fluidité
    groups[d] = MarkerCluster(name=label).add_to(m)

for i, row in df.iterrows():
    d_key = (row['year'] // 10) * 10 if row['year'] != 9999 else 9999
    img_b64 = get_base64_image(row['File name'])
    img_html = f'<img src="{img_b64}" style="width:100%; border-radius:4px; margin-bottom:12px;">' if img_b64 else ''

    # Popup avec charte graphique (Garamond pour l'OCR, Century Gothic pour le reste)
    popup_html = f"""
    <div style="width:280px; font-family: 'Century Gothic', 'Segoe UI', sans-serif; color: #333;">
        <h4 style="color:{COLOR_GREEN}; margin:0 0 10px 0; font-size:15px; font-weight: bold;">{row['File name']}</h4>
        {img_html}
        <div style="font-family: 'Garamond', 'Georgia', serif; font-style: italic; font-size: 16px; margin-bottom: 12px; line-height:1.2; border-left: 3px solid {COLOR_GREEN}; padding-left: 8px;">
            "{row.get('trans_ocr_text', '')}"
        </div>
        <div style="font-size: 12px; line-height: 1.6;">
            <b style="color:{COLOR_GREEN};">Date :</b> {row['Dates']}<br>
            <b style="color:{COLOR_GREEN};">Commune :</b> {row.get('Nom géographique', 'N/A')}<br>
            <b style="color:{COLOR_GREEN};">Lieu-dit :</b> {row.get('trans_hamlet_uniformise', 'N/A')}<br>
            <b style="color:{COLOR_GREEN};">Edifice :</b> {row.get('Édifice', 'N/A')}<br>
            <b style="color:{COLOR_GREEN};">Monument :</b> {row.get('trans_monument_uniformise', 'N/A')}
            <hr style="border: 0; border-top: 1px solid #ddd; margin: 10px 0;">
            <span style="font-size: 10px; color: #777;">
                <i>Précision : {row.get('source_georef', 'commune')}</i><br>
                <i>Confiance : {row.get('trans_confidence', 'N/A')}</i>
            </span>
        </div>
    </div>
    """

    color = COLOR_GREEN if 'monument' in str(row.get('source_georef', '')).lower() else COLOR_ORANGE

    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color='white', icon_color=color, icon='image', prefix='fa')
    ).add_to(groups[d_key])

# --- 6. FINALISATION ---
# Menu de sélection des calques (trié)
folium.LayerControl(collapsed=False).add_to(m)

# Logo du Vaucluse
# --- AJOUT DU LOGO VAUCLUSE (BAS À GAUCHE) ---

logo_url = "https://upload.wikimedia.org/wikipedia/fr/thumb/6/61/Logo-dept-vaucluse-2025.svg/500px-Logo-dept-vaucluse-2025.svg.png?20250602151636"

logo_html = f"""
<div style="
    position: fixed;
    bottom: 50px; left: 20px; width: 120px; height: auto;
    z-index: 9999;
    background-color: rgba(255, 255, 255, 0.8);
    padding: 10px;
    border: 2px solid {COLOR_GREEN};
    border-radius: 5px;
    box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
    ">
    <img src="{logo_url}" style="width: 100%;">
</div>
"""

m.get_root().html.add_child(folium.Element(logo_html))

m.save(output_html)
print(f"✅ Carte générée avec succès : {output_html}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Erreur chargement contours : 'geometry'
✅ Carte générée avec succès : carte_vaucluse_interactive.html


Il y a un problème dans l'importation des tracés des frontières départementales et communales, mais la carte s'affiche quand même.